In [1]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np

from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
import matplotlib.pyplot as plt
import json


Using TensorFlow backend.


In [2]:
###change data set into [('old', 'new'), (...]
json_filename = '/Users/yihe/Desktop/Stanford/yi_can_summarize/data/cnn/data.json'
sizes_of_text, sizes_of_summary= [] , []
with open(json_filename, 'r') as f:
    data = json.load(f)
all_data = []
i=0
for key in data.keys():
    if i > 1000:
        break
    if len(data[key])>1:
        all_data.append((data[key][0].strip(), data[key][1].strip()))
        
        size_of_text = len(data[key][0].strip().split(' '))
        size_of_summary = len(data[key][1].strip().split(' '))
        sizes_of_text.append(size_of_text)
        sizes_of_summary.append(size_of_summary)
        i+=1


In [3]:
Tx, Ty = int(np.percentile(sizes_of_text, 75)), int(np.percentile(sizes_of_summary, 75))
print Tx, Ty

76 14


In [4]:
m = len(all_data)
print m

1001


In [6]:
## change text into {'word':indx}
def create_vocab_list(vocab_text):
    vocab_list={}
    with open(vocab_text, 'r') as vocab_f:
        i=0
        for line in vocab_f:
            vocab_list[line.split(' ')[0]]=i
            i+=1
    
    return vocab_list

human_vocab_file = "/Users/yihe/Desktop/Stanford/yi_can_summarize/data/cnn/human_vocab.txt"
machine_vocab_file = "/Users/yihe/Desktop/Stanford/yi_can_summarize/data/cnn/machine_vocab.txt"

human_vocab, machine_vocab = create_vocab_list(human_vocab_file), create_vocab_list(machine_vocab_file)
inv_machine = dict(enumerate(sorted(machine_vocab)))

In [7]:
print "number of vocab in human vocab {}".format(len(human_vocab.keys()))
print "number of vocab in machine vocab {}".format(len(machine_vocab.keys()))

machine_vocab['<UNK>']

number of vocab in human vocab 87199
number of vocab in machine vocab 47799


47797

In [8]:
def preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty):
    
    X, Y = zip(*dataset)
    
    X = np.array([string_to_int_X(i, Tx, human_vocab) for i in X])
    Y = [string_to_int_Y(t, Ty, machine_vocab) for t in Y]
    
    return X, np.array(Y)

def string_to_int_X(string, length, vocab):
    """
    Converts all strings in the vocabulary into a list of integers representing the positions of the
    input string's characters in the "vocab"
    
    Arguments:
    string -- input string, e.g. 'Wed 10 Jul 2007'
    length -- the number of time steps you'd like, determines if the output will be padded or cut
    vocab -- vocabulary, dictionary used to index every character of your "string"
    
    Returns:
    rep -- list of integers (or '<unk>') (size = length) representing the position of the string's character in the vocabulary
    """
    
    #make lower to standardize
    string = string.split(' ')

    if len(string) > length:
        string = string[:length]

    rep = list(map(lambda x: vocab.get(x,vocab['<UNK>']), string))

    if len(string) < length:
        rep += [vocab['<PAD>']] * (length - len(string))
    
    rep = [[i] for i in rep]

    #print (rep)
    return rep
def string_to_int_Y(string, length, vocab):
    """
    Converts all strings in the vocabulary into a list of integers representing the positions of the
    input string's characters in the "vocab"
    
    Arguments:
    string -- input string, e.g. 'Wed 10 Jul 2007'
    length -- the number of time steps you'd like, determines if the output will be padded or cut
    vocab -- vocabulary, dictionary used to index every character of your "string"
    
    Returns:
    rep -- list of integers (or '<unk>') (size = length) representing the position of the string's character in the vocabulary
    """
    
    #make lower to standardize
    string = string.split(' ')

    if len(string) > length:
        string = string[:length]

    rep = list(map(lambda x: vocab.get(x,vocab['<UNK>']), string))

    if len(string) < length:
        rep += [vocab['<PAD>']] * (length - len(string))
    
    #rep = [[i] for i in rep]

    #print (rep)
    return rep
   
   

In [9]:
X, Y = preprocess_data(all_data, human_vocab, machine_vocab, Tx, Ty)

In [10]:
print("X.shape:", X.shape)
print("Y.shape:", Y.shape)


('X.shape:', (1001, 76, 1))
('Y.shape:', (1001, 14))


In [11]:
def softmax(x, axis=1):
    """Softmax activation function.
    # Arguments
        x : Tensor.
        axis: Integer, axis along which the softmax normalization is applied.
    # Returns
        Tensor, output of softmax transformation.
    # Raises
        ValueError: In case `dim(x) == 1`.
    """
    ndim = K.ndim(x)
    if ndim == 2:
        return K.softmax(x)
    elif ndim > 2:
        e = K.exp(x - K.max(x, axis=axis, keepdims=True))
        s = K.sum(e, axis=axis, keepdims=True)
        return e / s
    else:
        raise ValueError('Cannot apply softmax to a tensor that is 1D')

In [12]:
# Defined shared layers as global variables
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)

In [13]:
def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attetion) LSTM cell
    """
    
    ### START CODE HERE ###
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a" (≈ 1 line)
    s_prev = repeator(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis (≈ 1 line)
    concat = concatenator([a,s_prev])
    # Use densor1 to propagate concat through a small fully-connected neural network to compute the "intermediate energies" variable e. (≈1 lines)
    e = densor1(concat)
    # Use densor2 to propagate e through a small fully-connected neural network to compute the "energies" variable energies. (≈1 lines)
    energies = densor2(e)
    # Use "activator" on "energies" to compute the attention weights "alphas" (≈ 1 line)
    alphas =  activator(energies)
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell (≈ 1 line)
    context = dotor([ alphas,a])
    ### END CODE HERE ###
    
    return context

In [40]:
n_a = 32
n_s = 64
post_activation_LSTM_cell = LSTM(n_s, return_state = True)
output_layer = Dense(len(machine_vocab), activation=softmax)
Input(shape=(n_s,), name='s0')

<tf.Tensor 's0_2:0' shape=(?, 14) dtype=float32>

In [41]:
output_layer

In [49]:
def model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    """
    Arguments:
    Tx -- length of the input sequence
    Ty -- length of the output sequence
    n_a -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM
    human_vocab_size -- size of the python dictionary "human_vocab"
    machine_vocab_size -- size of the python dictionary "machine_vocab"

    Returns:
    model -- Keras model instance
    """
    
    # Define the inputs of your model with a shape (Tx,)
    # Define s0 and c0, initial hidden state for the decoder LSTM of shape (n_s,)
    X = Input(shape=(Tx, human_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    #output0=Input(shape=(m))
    s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
    
    ### START CODE HERE ###
    
    # Step 1: Define your pre-attention Bi-LSTM. Remember to use return_sequences=True. (≈ 1 line)
    a = Bidirectional(LSTM(n_a, return_sequences = True), input_shape = (m, Tx, n_a*2))(X)
    
    # Step 2: Iterate for Ty steps
    for t in range(Ty):
    
        # Step 2.A: Perform one step of the attention mechanism to get back the context vector at step t (≈ 1 line)
        context = one_step_attention(a, s)
        
        # Step 2.B: Apply the post-attention LSTM cell to the "context" vector.
        # Don't forget to pass: initial_state = [hidden state, cell state] (≈ 1 line)
        s, _, c =  post_activation_LSTM_cell(context,initial_state = [s, c])
        
        # Step 2.C: Apply Dense layer to the hidden state output of the post-attention LSTM (≈ 1 line)
        out = output_layer(s)
        
        # Step 2.D: Append "out" to the "outputs" list (≈ 1 line)
        outputs.append(out)
    
    # Step 3: Create model instance taking three inputs and returning the list of outputs. (≈ 1 line)
    model = Model([X, s0, c0], outputs = outputs)
    
    ### END CODE HERE ###
    
    return model

In [50]:
model = model(Tx, Ty, n_a, n_s, 1, 1)

ValueError: Input 0 is incompatible with layer dense_1: expected axis -1 of input shape to have value 128 but got shape (None, 76, 78)

In [47]:
print Tx, Ty, n_a, n_s

76 14 32 14


In [17]:
opt = Adam(lr = 0.005, beta_1 = 0.9, beta_2 = 0.999,decay = 0.01)  
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = opt,metrics = ['accuracy']) 

In [39]:
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(Y.swapaxes(0,1))

s0.shape
c0.shape
len(outputs[0])

1001

In [19]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 76, 1)        0                                            
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 64)           0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 76, 64)       8704        input_1[0][0]                    
__________________________________________________________________________________________________
repeat_vector_1 (RepeatVector)  (None, 76, 64)       0           s0[0][0]                         
                                                                 lstm_1[0][0]                     
          

In [20]:
model.fit([X, s0, c0], outputs, epochs=1, batch_size=1000)

Epoch 1/1
1001/1001 [==============================] - 26s 26ms/step - loss: 150.8336 - dense_3_loss: 10.7689 - dense_3_acc: 0.0000e+00 - dense_3_acc_1: 0.0000e+00 - dense_3_acc_2: 0.0000e+00 - dense_3_acc_3: 0.0000e+00 - dense_3_acc_4: 0.0000e+00 - dense_3_acc_5: 0.0000e+00 - dense_3_acc_6: 0.0000e+00 - dense_3_acc_7: 0.0000e+00 - dense_3_acc_8: 0.0000e+00 - dense_3_acc_9: 0.0000e+00 - dense_3_acc_10: 9.9900e-04 - dense_3_acc_11: 9.9900e-04 - dense_3_acc_12: 9.9900e-04 - dense_3_acc_13: 9.9900e-04


In [21]:
EXAMPLES = ['it cost more than to develop and doesn t come with French fries on the side',
            'the response to the increasing natural disaster in pakistan is like so much else when it comes to American relations with that country too little and too tentative',
            'seven Republican presidential contenders faced off Monday in one of the first debates of the primary season, offering policy ideas and criticism of president']


In [23]:
for example in EXAMPLES:   
    source = np.array([string_to_int_X(example, Tx, human_vocab)])
    prediction = model.predict([source, s0, c0])
    prediction = np.argmax(prediction, axis = -1)
    output = [inv_machine[int(i)] for i in prediction]

    print "source: {}".format(example)
    print"output: {}".format(output)

source: it cost more than to develop and doesn t come with French fries on the side
output: ['zynga', 'zynga', 'zynga', 'zynga', 'zynga', 'zynga', 'zynga', 'zynga', 'zynga', 'zynga', 'zynga', 'zynga', 'zynga', 'zynga']
source: the response to the increasing natural disaster in pakistan is like so much else when it comes to American relations with that country too little and too tentative
output: ['zynga', 'zynga', 'zynga', 'zynga', 'zynga', 'zynga', 'zynga', 'zynga', 'zynga', 'zynga', 'zynga', 'zynga', 'zynga', 'zynga']
source: seven Republican presidential contenders faced off Monday in one of the first debates of the primary season, offering policy ideas and criticism of president
output: ['zynga', 'zynga', 'zynga', 'zynga', 'zynga', 'zynga', 'zynga', 'zynga', 'zynga', 'zynga', 'zynga', 'zynga', 'zynga', 'zynga']
